In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 10000)
import numpy as np
import string
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW
from SoundsLike.SoundsLike import Search
import Levenshtein

In [3]:
df = pd.read_csv("../../data/conll03/train.txt", sep=" ")
#df = pd.read_csv("data/dev.txt", sep=" ")
#df = pd.read_csv("data/test.txt", sep=" ")

In [4]:
df.head()

,-DOCSTART-,-X-,-X-.1,O
0,EU,NNP,B-NP,B-ORG
1,rejects,VBZ,B-VP,O
2,German,JJ,B-NP,B-MISC
3,call,NN,I-NP,O
4,to,TO,B-VP,O


In [5]:
df = df[df['O'].notna()]

In [6]:
df.drop(['-X-', '-X-.1'], axis=1, inplace=True)

In [7]:
df.columns = ['Word', 'Tag']

In [8]:
df.head()

,Word,Tag
0,EU,B-ORG
1,rejects,O
2,German,B-MISC
3,call,O
4,to,O


In [9]:
#df.drop(df[df['Word'] == "-DOCSTART-"].index, axis=0, inplace=True)

In [10]:
df['Tag'].unique()

array(['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG',
       'I-MISC', 'I-LOC'], dtype=object)

In [11]:
tags = df['Tag'].values
words = df['Word'].values

In [12]:
new_tags = []
for t in tags:
    if t == "B-ORG":
        new_tags.append("ORG")
    elif t == "B-PER":
        new_tags.append("PER")
    elif t == "I-PER":
        new_tags.append("PER")
    elif t == "B-LOC":
        new_tags.append("LOC")
    elif t == "I-ORG":
        new_tags.append("ORG")
    elif t == "I-LOC":
        new_tags.append("LOC")
    else:
        new_tags.append("O")

In [13]:
df.drop(['Tag'], axis=1, inplace=True)
df.drop(['Word'], axis=1, inplace=True)

In [14]:
df['Tag'] = new_tags
df['Word'] = words

In [15]:
len(df)

202388

In [16]:
sentence_no = 0
sentence = []
for w in words:
    sentence.append(sentence_no)
    if w == ".":
        sentence_no = sentence_no + 1

In [17]:
df.insert(0, 'Sentence #', sentence)

In [18]:
df.head(100)

,Sentence #,Tag,Word
0,0,ORG,EU
1,0,O,rejects
2,0,O,German
3,0,O,call
4,0,O,to
5,0,O,boycott
6,0,O,British
7,0,O,lamb
8,0,O,.
9,1,PER,Peter


In [26]:
dataset = []
count = 0
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    error = []
    try:
        
        x = Search.perfectHomophones(str(word))
        if len(x) > 1:
            count = count + 1
        for d in x:
            err = Levenshtein.distance(str(word), d)
            error.append(err)
            word = x[error.index(max(error))]
    except ValueError:
        pass 
    dataset.append((sentence_no, tag, word))
df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [31]:
df.head(100)

,Sentence #,Tag,Word
0,0,ORG,Eu
1,0,O,Rejects
2,0,O,Jerman
3,0,O,Kaul
4,0,O,To
5,0,O,Boycott
6,0,O,British
7,0,O,Lahm
8,0,O,.
9,1,PER,Pieter


In [29]:
g_test = df.groupby("Sentence #")
x = pd.DataFrame({"Sentence": g_test.apply(lambda sdf: " ".join(map(str,sdf.Word))),
                       "Tag": g_test.apply(lambda sdf: ",".join(map(str,sdf.Tag)))})

In [30]:
x.head()

,Sentence,Tag
Sentence #,,
0,Eu Rejects Jerman Kaul To Boycott British Lahm .,"ORG,O,O,O,O,O,O,O,O"
1,Pieter Blackburn Brussels 1996-08-22 The European Commision Said Aune Thursday It Disagreed With Jerman Advice To Consumer'S To Shun British Lahm Until Scientists Determine Whether Mad Kough Disease Can Be Transmitted To Sheep .,"PER,PER,LOC,O,O,ORG,ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,Germany Es Representative To The European Union Es Veterinary Commitee Werner Zwingmann Said Aune Wednesday Consumer'S Schuld Bae sheepmeat From Country'S Other Than Brittain Until The Scientific Advice Was Clearer .,"LOC,O,O,O,O,ORG,ORG,O,O,O,PER,PER,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O"
3,"Wii Deux n't Support Any Such Recommendation Because Wiehe Deux n't Sieh Any Grounds For It , The Commision Es Chief Spokesman Nicolaus Van Dirr Paz Tolled A News Briefing .","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,O,O,O,PER,PER,PER,PER,O,O,O,O,O"
4,Hee Said Further Scientific Study Was Required And If It Was Found That Action Was Needed It Schuld Be Taken Bae The European Union .,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,ORG,ORG,O"


In [ ]:
import re

def decontracted(phrase):
    # general
    phrase = re.sub(r" \'s", "\'s", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" n't", "n't", phrase)
    phrase = re.sub(r" \'re", "\'re", phrase)
    phrase = re.sub(r" \'d", "\'d", phrase)
    phrase = re.sub(r" \'ll", "\'ll", phrase)
    phrase = re.sub(r" \'t", "\'t", phrase)
    phrase = re.sub(r" \'ve", "\'ve", phrase)
    phrase = re.sub(r" \'m", "\'m", phrase)
    return phrase

In [ ]:
p_data = []
sentence_no = 0
new_df = []
for sentence, tags in zip(x['Sentence'].values.tolist(), x['Tag'].values.tolist()):
    words = sentence.split(" ")
    ind = [ i for i, word in enumerate(words) if word == "'s" or word == "'t" or word == "'re" or word == "'d" or word == "'ll" or word == "'t" or word == "'ve" or word == "'m" or word == "n't"]
    labels = tags.split(",")
    sentence = " ".join(words)
    sentence = decontracted(sentence)
    tags = [ elem for i, elem in enumerate(labels) if i not in ind]
    stripped = sentence.translate(str.maketrans('', '', string.punctuation))
    stripped = stripped.split(" ")
    for word, label in zip(stripped, tags):
        p_data.append((sentence_no, word, label))
    sentence_no = sentence_no + 1

In [ ]:
df = pd.DataFrame(p_data, columns=['Sentence #', 'Word', 'Tag'])

In [ ]:
len(df)

In [ ]:
dataset = []
for sentence_no, word, tags in zip(df['Sentence #'].values.tolist(), df['Word'].values.tolist(), df['Tag'].values.tolist()):
    if word != "":
        dataset.append((sentence_no, word, tags))
df = pd.DataFrame(dataset, columns=['Sentence #', 'Word', 'Tag'])

In [ ]:
df.head(200)

In [ ]:
entities = list(set(df[df['Tag'] != "O"]['Sentence #'].values.tolist()))

In [ ]:
import random
randomlist = random.sample(entities, int(len(entities)/2))
randomlist.sort()

In [ ]:
randomlist

In [ ]:
dataset = []
for sentence_no, tag, word in zip(df['Sentence #'].values.tolist(), df['Tag'].values.tolist(), df['Word'].values.tolist()):
    if sentence_no in randomlist:
        if tag != "O":
            if len(word) >=4:
                word = list(word)
                ind = random.randint(2,len(word)-1)
                temp = word[ind-1]
                word[ind-1] = word[ind]
                word[ind] = temp
                word = "".join(word)
                dataset.append((sentence_no, tag, word))
            else:
                dataset.append((sentence_no, tag, word))
        else:
            dataset.append((sentence_no, tag, word))
    else:
        dataset.append((sentence_no, tag, word))

In [ ]:
len(dataset)

In [ ]:
new_df = pd.DataFrame(dataset, columns=['Sentence #', 'Tag', 'Word'])

In [ ]:
new_df.head(20)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_whole_word_mask=True)

In [ ]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(new_df)
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
labels = [[s[1] for s in sentence] for sentence in getter.sentences]

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(str(word))
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(sentences, labels)]
#tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
#labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
tokenized_texts[0:50]

In [ ]:
for sentence in tokenized_texts:
    print(sentence)
    break

In [ ]:
i = 0
sentence_no = 0
new_sentence = []
new_data = []
for sentence, label in zip(tokenized_texts, labels):
    new_tokens = []
    new_tags = []
    for token, tag in zip(sentence, label):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_tokens.append(token)
            new_tags.append(tag)
    for new_token, new_tag in zip(new_tokens, new_tags):
        if new_token in string.punctuation:
            continue
        else:
            new_data.append((sentence_no, new_token, new_tag))
    sentence_no = sentence_no + 1

In [ ]:
new_data[0:50]

In [ ]:
data = pd.DataFrame(new_data, columns=['Sentence #', 'Word', 'Tag'])

In [32]:
df.to_csv("../../data/conll03/new_conll_train_preprocessed_with_punctuation_with_all_broken_entities2.csv")